## import library and set some variable

In [1]:
from selenium import webdriver # act like a web browser
from bs4 import BeautifulSoup # extract information from 
from selenium.common.exceptions import WebDriverException # import unclickable exception
import time # slow down the algorithm
import re
scroll_pause_time = 0.5
browser_wait_time = 10
divided_scroll_height = 60
hotel_id_in_our_database = 0

## cre8 the data object

In [2]:
data_vary = dict(hotel_id_in_our_database = [],hotel_user_Ratings = [],hotel_user_Ratings_text = [],
                hotel_room_Type_Names = [],hotel_room_Prices = [],hotel_room_Currency = [],hotel_room_Benefits = [],
                hotel_room_Capacities = [],feature_data = [],hotel_room_Prices_standard = []) # just cre8 your own hotel id
data_static = dict(hotel_id_in_our_database = [],hotel_name = [],hotel_location = [],hotel_star = [],
                   hotel_property_highlights = [],hotel_location_rating = [],hotel_location_level = [],
                  hotel_nearby_place_distance = [],hotel_popular_place_distance = [])

## access the agoda web

In [3]:
chrome_driver_path_com9 = r'C:\Users\5842005226\Desktop\chromedriver.exe'
chrome_driver_path_home = r'C:\Users\Paem\Desktop\chromedriver.exe'
chrome_driver_path_pudit = r'/Users/pudit/Downloads/chromedriver'
driver = webdriver.Chrome(chrome_driver_path_pudit) # choose one path
# change the link here
agoda_link = 'https://www.agoda.com/pages/agoda/default/DestinationSearchResult.aspx?asq=zWuVSTFwAmUZtJhrjzSYy1wsTkjMQWoz5vdhcDTo6aet9GMrbwvSAO7YZ1OsQyBVq9i2BYOtnR9%2FSmFuCedBLTM9cgxZrrJkUlDRY5VuCfWf%2B9BcBaWUWO%2F0puG%2B%2F8u3dBe7G%2BCV0mlTWwX03LR6dung8R99uT3pYtroEe9EvrzHMPTvy434rjKcPr%2FVqayItVnKWhZ24jrVsMFUTxZOiJpxj%2FA%2BylTfAGs1qJCjm9kLAZGKtabHNwvy6CSwhhhP&city=9395&cid=1732641&tick=636861849226&languageId=22&userId=81c4cc68-ad7d-4872-bd15-505dbe030965&sessionId=10c1gqc3kwqdoc4q1rdxkge0&pageTypeId=1&origin=TH&locale=th-TH&tag=5bd24d74-c5f5-b6ce-4e9c-a2f2c7d0d66c&gclid=EAIaIQobChMIx5OS_KjH4AIViIRwCh2DrwjhEAAYASAAEgJ64vD_BwE&aid=81837&currencyCode=THB&htmlLanguage=th-th&cultureInfoName=th-TH&ckuid=81c4cc68-ad7d-4872-bd15-505dbe030965&prid=0&checkIn=2019-03-19&checkOut=2019-03-20&rooms=1&adults=2&children=0&priceCur=THB&los=1&textToSearch=%E0%B8%81%E0%B8%A3%E0%B8%B8%E0%B8%87%E0%B9%80%E0%B8%97%E0%B8%9E&travellerType=1&productType=1&sort=agodaRecommended'
driver.get(agoda_link)

## cre8 the function that scroll the page until specific element found

In [4]:
def scroll_page_till_find(scroll_pause_time,last_height,scroll_height,element,driver):
    while True:
        first_height = last_height
        last_height += scroll_height
        driver.execute_script("window.scrollTo(" + str(first_height) + ", " + str(last_height) + ")")
        first_height = last_height
        time.sleep(scroll_pause_time)
        if abs(last_height - element.location['y']) <= scroll_height*2:
            break
    return last_height

## cre8 the function to collect hotel static data

In [5]:
def collect_hotel_static_data(soup):
    try: # handle unknown web structure
        hotel_property_highlights = ''
        hotel_name = soup.find('h1',{'class':"FirstTileContent__Title"}).get_text()
        hotel_location = soup.find('div',{'class':"FirstTileContent__Address"}).get_text()
        hotel_star_string = soup.find('i',{'data-selenium':"mosaic-hotel-rating"})['class'][3]
        hotel_star = int(re.findall('.*-([0-9]*)',hotel_star_string)[0])
        hotel_star = hotel_star/10 if hotel_star > 5 else hotel_star # correct the star
        hotel_property_highlights_list = soup.find_all('li',{'class':"fav-features__listitem fav-features__listitem__4 qa-fav-features__listitem"})
        for hotel_property_highlight in hotel_property_highlights_list:
            hotel_property_highlight_text = hotel_property_highlight.find('p',{'class':"fav-features__text fav-features__text--small"}).get_text()
            hotel_property_highlights += hotel_property_highlight_text + '&&'
        return [hotel_name,hotel_location,hotel_star,hotel_property_highlights]
    except ValueError:
        return [None,None,None,None]

## cre8 the function to collect hotel dynamic data

In [6]:
def collect_hotel_vary_data(soup,hotel_room_type_info):
    hotel_user_Ratings = []
    hotel_user_Ratings_text = []
    hotel_room_Type_Names = []
    hotel_room_Prices = []
    hotel_room_Prices_standard = []
    hotel_room_Currency = []
    hotel_room_Benefits = []
    hotel_room_Capacities = []
    hotel_user_rating = float(soup.find('div',{'class':"ReviewScoreCompact__score"}).span.get_text())
    hotel_user_rating_text = soup.find('div',{'class':"ReviewScoreCompact__text"}).div.get_text()
    hotel_room_type_span = hotel_room_type_info.find('span',{'data-selenium':'masterroom-title-name'})
    hotel_room_soldout_span = hotel_room_type_info.find('span',{'class':"SoldOutRoom-masterRoomName"})
    if hotel_room_type_span is not None: # check whether hotel_room_type obj is normal or not
        hotel_id = hotel_room_type_info['id'] # use in feature finding function
        hotel_room_type_name = hotel_room_type_span.get_text() # get room type text
        hotel_room_type_info_prices_benefits = hotel_room_type_info.find('div',{'class':'ChildRoomsList'}).contents # get all prices & bentfit in each hotel_room_type 
        for hotel_room_type_info_price_benefit in hotel_room_type_info_prices_benefits[1:]:
            hotel_room_type_benefits = hotel_room_type_info_price_benefit.find('div',{'class':"ChildRoomsList-room-featurebucket ChildRoomsList-room-featurebucket-Benefits"}).contents # find all benefits
            hotel_room_type_price = hotel_room_type_info_price_benefit.find('div',{'class':'PriceContainer'})# find the price
            hotel_room_benefits = ''
            for hotel_room_type_each_benefit in hotel_room_type_benefits[1:]: #find all benefits except 'benefit' head column
                hotel_room_benefits += hotel_room_type_each_benefit.find('span',{'class':"ChildRoomsList-roomFeature-TitleWrapper"}).get_text() + ';'
            hotel_room_capacity_icons = hotel_room_type_info_price_benefit.find('div',{'class':"ChildRoomsList-capacity"}).span # find icon element
            hotel_room_capacity = dict(adults = 0,children = 0) # store the room capacity as dict
            hotel_room_capacity = find_room_capacity_from_icon(hotel_room_capacity_icons,hotel_room_capacity) # apply the function on icon element to find the room capacity
            additional_hotel_room_capacity_icon = hotel_room_capacity_icons.next_sibling
            try:
                additional_hotel_room_capacity_icon_class = ' '.join(additional_hotel_room_capacity_icon['class'])
                if 'icon-hide' not in additional_hotel_room_capacity_icon_class and 'age-range' not in additional_hotel_room_capacity_icon_class :
                    hotel_room_capacity['children'] += int(additional_hotel_room_capacity_icon.find('span',class_ = "child-num").get_text())
            except TypeError:
                pass
            if hotel_room_type_price.find('div',{'class':"CrossedOutPrice "}) is None:
                hotel_room_price_str = hotel_room_type_price.find('div',{'class':'finalPrice'}).find('strong',{'data-ppapi':"room-price"}).get_text() # extract room price
                hotel_room_price = int(''.join(hotel_room_price_str.split(',')))
                hotel_room_price_standard = hotel_room_price
            else:
                hotel_room_price_standard = float(hotel_room_type_price.find('div',{'class':"CrossedOutPrice "})["data-element-cor"])
                hotel_room_price = float(hotel_room_type_price.find('div',{'class':"CrossedOutPrice "})["data-room-price"])
            hotel_room_price_currency = hotel_room_type_price.find('div',{'class':'finalPrice'}).find('span',{'class':"pd-currency"}).get_text() # extract currency
            hotel_user_Ratings.append(hotel_user_rating)
            hotel_room_Type_Names.append(hotel_room_type_name)
            hotel_room_Prices.append(hotel_room_price)
            hotel_room_Currency.append(hotel_room_price_currency)
            hotel_room_Benefits.append(hotel_room_benefits)
            hotel_user_Ratings_text.append(hotel_user_rating_text)
            hotel_room_Capacities.append(hotel_room_capacity)
            hotel_room_Prices_standard.append(hotel_room_price_standard)
        return [hotel_id,hotel_user_Ratings,hotel_user_Ratings_text,hotel_room_Type_Names,hotel_room_Prices,hotel_room_Currency,hotel_room_Benefits,hotel_room_Capacities,hotel_room_Prices_standard]
    elif hotel_room_soldout_span is not None: #check whether hotel_room_typ is sold out or not
        hotel_user_Ratings.append(hotel_user_rating)
        hotel_room_Type_Names.append(None)
        hotel_room_Prices.append(None)
        hotel_room_Currency.append(None)
        hotel_room_Benefits.append(None)
        hotel_room_Capacities.append(None)
        hotel_room_Prices_standard.append(None)
        hotel_user_Ratings_text.append(hotel_user_rating_text)
        return [None,hotel_user_Ratings,hotel_user_Ratings_text,hotel_room_Type_Names,hotel_room_Prices,hotel_room_Currency,hotel_room_Benefits,hotel_room_Capacities,hotel_room_Prices_standard]
    else:
        return None

## cre8 the function to find room capacity from icon

In [7]:
def find_room_capacity_from_icon(hotel_room_capacity_icons,hotel_room_capacity):
    try: # consider that : icon is extra bed or not
        icon_class_str = ' '.join(hotel_room_capacity_icons['class']) # find the class of that icon element
        hotel_room_capacity = SubFunction_find_room_capacity_from_icon(hotel_room_capacity_icons,icon_class_str,hotel_room_capacity) # use the function to find people_type and number_of_people
        hotel_room_capacity['extra_bed'] = 0 # no extra bed
    except KeyError: # if icon element is extra bed -> do this
        hotel_room_capacity['extra_bed'] = 1 # have extra bed
        for hotel_room_capacity_icons_element in hotel_room_capacity_icons.contents:
            icon_class_str = ' '.join(hotel_room_capacity_icons_element['class']) # find the class of all icon elements
            if 'icon-hide' not in icon_class_str: # if not-nide not in icon -> do this
                hotel_room_capacity = SubFunction_find_room_capacity_from_icon(hotel_room_capacity_icons_element,icon_class_str,hotel_room_capacity) # use the function to find people_type and number_of_people
    return hotel_room_capacity

In [8]:
def SubFunction_find_room_capacity_from_icon(hotel_room_capacity_icons,icon_class_str,hotel_room_capacity):
    if 'num' in icon_class_str: # if icon pattern is 3 x icon or icon pattern is additional class
        num = int(hotel_room_capacity_icons.span.get_text()) # get number of people
        considered_class = ' '.join(hotel_room_capacity_icons.i['class'])
        p_type = re.findall('ficon ficon-(.*)-one .*',considered_class).pop() # get type of people
        hotel_room_capacity[p_type] += num # update the room capacity data
    else: # if icon pattern is icon+icon
        for hotel_room_capacity_icon in hotel_room_capacity_icons.contents:
            considered_class = ' '.join(hotel_room_capacity_icon['class']) # find the class of each icon
            if 'icon-hide' not in considered_class and 'plus' not in considered_class: # if that icon is not icon-hide and not plus object
                p_type = re.findall('ficon ficon-(.*)-one .*',considered_class).pop() # extract people type
                hotel_room_capacity[p_type] += 1 # update the room capacity data
    return hotel_room_capacity

## cre8 the function to collect room feature data

In [9]:
def find_hotel_feature(driver,hotel_id,room_type_element):
    feature_data = dict()
    room_type_element.click() # apply click on the element
    time.sleep(2)
    completed_raw_html = driver.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(completed_raw_html,'lxml') # extract the information
    driver.find_element_by_xpath('//div[@class="close-area qa-room-info-close-area room-info-close-area"]/span').click()# close the pop up    
    all_pop_up_element = soup.find_all('div',{'class':'ReactModalPortal'})
    index = 0
    while True:
        pop_up_element_consider = all_pop_up_element[index]
        if pop_up_element_consider.find('div',{'class':'details__info-area'}) is not None:
            # extract all feature in string
            feature_hotel_room_list = pop_up_element_consider.find('div',{'class':'details__info-area'}).contents
            for feature_hotel_room in feature_hotel_room_list:
                feature_room_mode = feature_hotel_room.p.get_text() # extract the feature mode
                feature_room_list_in_mode = feature_hotel_room.ul.contents
                feature_room_string = ''
                for feature_room_in_mode in feature_room_list_in_mode:
                    try:
                        c = ' '.join(feature_room_in_mode['class'])
                    except KeyError: # BedType structure isn't same as other
                        c = ' '.join(feature_room_in_mode.ul.li['class'])
                    if 'disabled' not in c: # if its not disabled -> add it in the data
                        if feature_hotel_room.find('fieldset') is not None:
                            feature_room_string += feature_room_in_mode.span.get_text() + '||'
                        else:
                            feature_room_string += feature_room_in_mode.span.get_text() + '&&'
                feature_data[feature_room_mode] = feature_room_string
            break
        index += 1
    return feature_data

## write the function to extract nearby place

In [10]:
def extract_nearby_popular_info(soup,driver,click):
    nearby_place_distance = []
    if click: # if click popular -> click
        # click
        driver.find_element_by_xpath('//div[@class="NearByLocationBox__Tab"]/div[2]').click()
        # retrieve new info
        time.sleep(1)
        each_hotel_raw_html = driver.execute_script("return document.documentElement.outerHTML") # scrape the data
        soup = BeautifulSoup(each_hotel_raw_html,'lxml')
    nearby_info = soup.find('div',{'class':'PropertyIntroSideContent'}).find('div',{'class':'NearByLocationBox'}).div.next_sibling # extract only Nearby part
    for each_nearby_location in nearby_info.contents:
        each_nearby_location_str = ''
        each_nearby_location_str += each_nearby_location.span.get_text() + ' && '
        each_nearby_location_str += each_nearby_location.span.next_sibling.get_text()
        nearby_place_distance.append(each_nearby_location_str)
    return nearby_place_distance

## cre8 the function that close the unused tab

In [11]:
def close_unused_tab(default_window_handle): # close the tab that is located on right side of the main tab
    handles = list(driver.window_handles)  
    assert len(handles) > 1
    handles.remove(default_window_handle)
    assert len(handles) > 0
    driver.switch_to_window(handles[0])
    driver.close()
    driver.switch_to_window(default_window_handle)

## cre8 the function that update the static data

In [12]:
def update_hotel_static_data(data_static,hotel_id_in_our_database,hotel_name,hotel_location,hotel_star,
                   hotel_property_highlights,hotel_location_rating,hotel_location_level,
                  hotel_nearby_place_distance,hotel_popular_place_distance):
    data_static["hotel_id_in_our_database"].append(hotel_id_in_our_database)
    data_static["hotel_name"].append(hotel_name)
    data_static["hotel_location"].append(hotel_location)
    data_static["hotel_star"].append(hotel_star)
    data_static["hotel_property_highlights"].append(hotel_property_highlights)
    data_static["hotel_location_rating"].append(hotel_location_rating)
    data_static["hotel_location_level"].append(hotel_location_level)
    data_static["hotel_nearby_place_distance"].append(hotel_nearby_place_distance)
    data_static["hotel_popular_place_distance"].append(hotel_popular_place_distance)
    return data_static

## cre8 the function that update the dynamic data

In [13]:
def update_hotel_dynamic_data(data_vary,hotel_id_in_our_database,hotel_user_Ratings,hotel_user_Ratings_text,
                hotel_room_Type_Names,hotel_room_Prices,hotel_room_Currency,hotel_room_Benefits,
                hotel_room_Capacities,feature_data,hotel_room_Prices_standard):
    data_vary["hotel_id_in_our_database"].extend(hotel_id_in_our_database)
    data_vary["hotel_user_Ratings"].extend(hotel_user_Ratings)
    data_vary["hotel_user_Ratings_text"].extend(hotel_user_Ratings_text)
    data_vary["hotel_room_Type_Names"].extend(hotel_room_Type_Names)
    data_vary["hotel_room_Prices"].extend(hotel_room_Prices)
    data_vary["hotel_room_Currency"].extend(hotel_room_Currency)
    data_vary["hotel_room_Benefits"].extend(hotel_room_Benefits)
    data_vary["hotel_room_Capacities"].extend(hotel_room_Capacities)
    data_vary["feature_data"].extend(feature_data)
    data_vary["hotel_room_Prices_standard"].extend(hotel_room_Prices_standard)
    return data_vary

## main program

In [24]:
try:
    driver.maximize_window()
except WebDriverException:
    pass
continue_scrape = True ## if continue scrape <- change this to TRUE
while True:    
    if not continue_scrape:
        default_window_handle = driver.current_window_handle
        #hotel_id_in_our_database = 0
        last_height = 0
        element_index = 0
        agoda_page_height = driver.execute_script("return document.body.scrollHeight") # get the original page height
        scroll_height = agoda_page_height/divided_scroll_height
        click_elements = driver.find_elements_by_xpath('//h3[@class="hotel-name"]') # get the original click element
    while True:
        last_height = scroll_page_till_find(scroll_pause_time,last_height,scroll_height,click_elements[element_index],driver)
        time.sleep(2) # wait for the data to be loaded
        click_elements_new = driver.find_elements_by_xpath('//h3[@class="hotel-name"]') # get the loaded data
        if len(click_elements_new) > len(click_elements): # if new data loaded -> update the data
            click_elements = click_elements_new # update the data
        try: # try to click the element
            click_elements[element_index].click() # click the element
        except WebDriverException: # if cannot -> close the popup
            driver.find_element_by_xpath('//div[@class="LeaveSitePopup-CloseArea"]/span').click() # close the popup
            click_elements[element_index].click() # click the element again
        time.sleep(browser_wait_time) # browser wait
        driver.switch_to_window(driver.window_handles[1]) # switch to next window
        time.sleep(browser_wait_time)
        each_hotel_raw_html = driver.execute_script("return document.documentElement.outerHTML") # scrape the data
        soup = BeautifulSoup(each_hotel_raw_html,'lxml')
        ########### collect all static data ################
        try: # figure out whether page is normal page or not
            hotel_name = hotel_name = soup.find('h1',{'class':"FirstTileContent__Title"}).get_text()
            if hotel_name not in set(data_static['hotel_name']): # if hotel_name not in data -> update it
                hotel_name,hotel_location,hotel_star,hotel_property_highlights = collect_hotel_static_data(soup) # collect static data on the top
                hotel_location_rating = float(soup.find('div',{'class':"MapCompact"}).find('div',{'class':"ReviewScore ReviewScore--outline badgeRating-ReviewScore"}).span.get_text())
                hotel_location_level = soup.find('div',{'class':"MapCompact"}).find('div',{'class':"badgeRating-ScoreText"}).get_text()
                popular_click = False
                hotel_nearby_place_distance = extract_nearby_popular_info(soup,driver,popular_click) # extract nearby infomation
                popular_click = True
                hotel_popular_place_distance = extract_nearby_popular_info(soup,driver,popular_click) # retrieve all information in "popular" button
                ## COLLECT THE STATIC DATA ##
                data_static = update_hotel_static_data(data_static,hotel_id_in_our_database,hotel_name,hotel_location,hotel_star,hotel_property_highlights,hotel_location_rating,hotel_location_level,hotel_nearby_place_distance,hotel_popular_place_distance)
                ########### collect al dynamic data (also static) ################
                hotel_all_room_type_info = soup.find('div',{'id':"property-room-grid-root"}).find('div',{'class':'RoomGrid-content'})
                if hotel_all_room_type_info.div['class'][0] != 'RoomGrid-searchTimeOut': # check whether 'all' hotel room is sold out or not
                    agoda_2page_height = driver.execute_script("return document.body.scrollHeight") # get the original page height
                    last_2page_height = 0
                    scroll_2page_height = agoda_page_height/divided_scroll_height
                    for hotel_room_type_info in hotel_all_room_type_info.contents: # 'all' room not sold out -> collect the data
                        all_hotel_vary_data = collect_hotel_vary_data(soup,hotel_room_type_info)
                        if all_hotel_vary_data is not None: # check whether it's recommened tab or not
                            hotel_id,hotel_user_Ratings,hotel_user_Ratings_text,hotel_room_Type_Names,hotel_room_Prices,hotel_room_Currency,hotel_room_Benefits,hotel_room_Capacities,hotel_room_Prices_standard = all_hotel_vary_data
                            if hotel_id is not None: # check whether hotel room is sold out or not
                                #<span data-selenium="masterroom-title-name">Twin Family Room</span>
                                room_type_element = driver.find_element_by_xpath('//div[@id="' + hotel_id + '"]//span[@data-selenium="masterroom-title-name"]')
                                last_2page_height = scroll_page_till_find(scroll_pause_time,last_2page_height,scroll_2page_height,room_type_element,driver)
                                feature_data = [find_hotel_feature(driver,hotel_id,room_type_element)]*len(hotel_user_Ratings)
                            else:
                                feature_data = [None]*len(hotel_user_Ratings)
                            hotel_ids_in_our_database = [hotel_id_in_our_database]*len(hotel_user_Ratings)
                            ## COLLECT THE DYNAMIC DATA ##
                            data_vary = update_hotel_dynamic_data(data_vary,hotel_ids_in_our_database,hotel_user_Ratings,hotel_user_Ratings_text,hotel_room_Type_Names,hotel_room_Prices,hotel_room_Currency,hotel_room_Benefits,hotel_room_Capacities,feature_data,hotel_room_Prices_standard)
        except AttributeError: # if the page is not normal -> just close it
            pass
        driver.switch_to_window(default_window_handle)
        close_unused_tab(default_window_handle)
        hotel_id_in_our_database += 1
        element_index += 1
        if element_index >= len(click_elements): # if scroll down till find the last element -> break
            continue_scrape = False
            break
    try: # try to click the next-page button
        next_element = driver.find_element_by_xpath('//div[@class="clearfix pagination-panel"]/div/button')
        scroll_page_till_find(scroll_pause_time,last_height,scroll_height,next_element,driver)
        next_element.click()
        time.sleep(browser_wait_time)
    except WebDriverException:
        break

/Users/pudit/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: use driver.switch_to.window instead
/Users/pudit/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:66: DeprecationWarning: use driver.switch_to.window instead
/Users/pudit/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use driver.switch_to.window instead
  
/Users/pudit/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use driver.switch_to.window instead
  


IndexError: list index out of range

## save the data

In [104]:
import pickle
pickle_out = open("data_static_18.pickle","wb")
pickle.dump(data_vary, pickle_out)
pickle_out.close()
pickle_out = open("data_vary_18.pickle","wb")
pickle.dump(data_static, pickle_out)
pickle_out.close()

## Test Code !! Do Not Run It !!

In [31]:
driver.switch_to_window(driver.window_handles[0])
close_unused_tab(driver.window_handles[0])

C:\Users\Paem\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.
C:\Users\Paem\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use driver.switch_to.window instead
  
C:\Users\Paem\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use driver.switch_to.window instead
  


In [1]:
from selenium import webdriver # act like a web browser
from bs4 import BeautifulSoup # extract information from 
from selenium.common.exceptions import WebDriverException # import unclickable exception
import time # slow down the algorithm
import re
chrome_driver_path_com9 = r'C:\Users\5842005226\Desktop\chromedriver.exe'
driver = webdriver.Chrome(chrome_driver_path_com9)
agoda_link = 'https://www.agoda.com/pages/agoda/default/DestinationSearchResult.aspx?city=9395&checkIn=2019-03-15&los=1&rooms=1&adults=2&children=0&cid=-218&languageId=1&userId=4f5832b5-d4c3-4f4e-b879-07304220c449&sessionId=1q5vnabumlnzls3avljlc5xf&pageTypeId=1&origin=TH&locale=en-US&aid=130589&currencyCode=USD&htmlLanguage=en-us&cultureInfoName=en-US&ckuid=4f5832b5-d4c3-4f4e-b879-07304220c449&prid=0&checkOut=2019-03-16&priceCur=USD&textToSearch=Bangkok&travellerType=1&productType=1&sort=agodaRecommended'
driver.get(agoda_link)

In [108]:
hotel_all_room_type_info = soup.find('div',{'id':"property-room-grid-root"}).find('div',{'class':'RoomGrid-content'}).contents
hotel_room_type_info = hotel_all_room_type_info[9] # choose room type
hotel_room_type_info_prices_benefits = hotel_room_type_info.find('div',{'class':'ChildRoomsList'}).contents[1:]
hotel_room_type_info_price_benefit = hotel_room_type_info_prices_benefits[0] # choose room_type_price_benefit
hotel_room_capacity_icons = hotel_room_type_info_price_benefit.find('div',{'class':"ChildRoomsList-capacity"}).span

In [109]:
hotel_room_capacity = dict(adults = 0,children = 0)
hotel_room_capacity = find_room_capacity_from_icon(hotel_room_capacity_icons,hotel_room_capacity)
additional_hotel_room_capacity_icon = hotel_room_capacity_icons.next_sibling
try:
    additional_hotel_room_capacity_icon_class = ' '.join(additional_hotel_room_capacity_icon['class'])
    if 'icon-hide' not in additional_hotel_room_capacity_icon_class and 'age-range' not in additional_hotel_room_capacity_icon_class :
        hotel_room_capacity['children'] += int(additional_hotel_room_capacity_icon.find('span',class_ = "child-num").get_text())
except TypeError:
    pass

In [2]:
import pickle
pickle_in = open("data_static.pkl","rb")
data_static = pickle.load(pickle_in)
pickle_in = open("data_vary.pkl","rb")
data_vary = pickle.load(pickle_in)

In [51]:
driver.find_element_by_xpath('//div[@class="clearfix pagination-panel"]/div/button').click()

In [81]:
'Tongpoon Hotel' in set(data_static['hotel_name'])

True

In [127]:
data_vary['feature_data'][313]

{'Bathroom and toiletries': 'Towels&&',
 'Bed types': '2 king beds&&',
 'Clothing and laundry': 'Closet&&Clothes rack&&',
 'Comforts': 'Air conditioning&&Heating&&Wake-up service&&',
 'Dining, drinking, and snacking': 'Refrigerator&&',
 'Entertainment ': 'Free Wi-Fi in all rooms!&&TV&&',
 'Features': '2 king beds&&Room size: 64 m²/689 ft²&&Balcony/terrace&&Shower&&Free Wi-Fi&&',
 'Guest ratings': '&&',
 'Layout and furnishings': 'Desk&&',
 'Safety and security features': 'In-room safe box&&'}

In [140]:
completed_raw_html = driver.execute_script("return document.documentElement.outerHTML")
soup = BeautifulSoup(completed_raw_html,'lxml') # extract the information
driver.find_element_by_xpath('//div[@class="close-area qa-room-info-close-area room-info-close-area"]/span').click()# close the pop up    
all_pop_up_element = soup.find_all('div',{'class':'ReactModalPortal'})


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="close-area qa-room-info-close-area room-info-close-area"]/span"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.15063 x86_64)


In [143]:
pop_up_element_consider = all_pop_up_element[7]
feature_hotel_room_list = pop_up_element_consider.find('div',{'class':'details__info-area'}).contents
feature_room_list_in_mode = feature_hotel_room_list[0].ul.contents

In [145]:
feature_room_in_mode = feature_room_list_in_mode[]

In [147]:
' '.join(feature_room_in_mode['class'])

'details__features-item'

In [150]:
driver.quit()

In [4]:
driver.switch_to_window(driver.window_handles[1])

C:\Users\Paem\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


In [5]:
completed_raw_html = driver.execute_script("return document.documentElement.outerHTML")
soup = BeautifulSoup(completed_raw_html,'lxml')
all_pop_up_element = soup.find_all('div',{'class':'ReactModalPortal'})

In [16]:
pop_up_element_consider = all_pop_up_element[7]
feature_hotel_room_list = pop_up_element_consider.find('div',{'class':'details__info-area'}).contents
for feature_hotel_room in feature_hotel_room_list:
    feature_room_list_in_mode = feature_hotel_room.ul.contents
    for feature_room_in_mode in feature_room_list_in_mode:
        try:
            print(' '.join(feature_room_in_mode['class']))
        except KeyError:
            print(' '.join(feature_room_in_mode.ul.li['class']))

details__features-item
details__features-item
details__features-item
details__features-item
details__features-item
details__features-item
details__features-item
details__bed-layout-item
details__amenities-item
details__amenities-item
details__amenities-item details__amenities-item_disabled
details__amenities-item
details__amenities-item
details__amenities-item
details__amenities-item
details__amenities-item
details__amenities-item
details__amenities-item
details__amenities-item
details__amenities-item
details__amenities-item
details__amenities-item
details__amenities-item
details__amenities-item details__amenities-item_disabled
details__rating-list-item


In [10]:
len(feature_hotel_room_list)

10

In [53]:
data_static['hotel_name']

['Solitaire Bangkok Sukhumvit 11',
 'Somerset Ekamai Bangkok',
 'MonkeyNap Hostel',
 'The Berkeley Hotel Pratunam',
 'The Bedroom Ladprao 101',
 'Nasa Vegas Hotel',
 "Harry's Suite",
 'Jellybean CheQinn Hostel Sukhumvit 4',
 'The Boss Sathorn Hotel',
 'SC Park Hotel',
 None,
 '48Metro Hotel Bangkok',
 'Sukhumvit 20 Hotel',
 'Wongcharearn Mansion',
 'De Botan Hotel',
 'B2 Bangkok Hotel - Srinakarin',
 'Bangkok 68',
 'Romance Hotel Bangna',
 'VX The Fifty Hostel',
 'Baron Zotel',
 'Parc Town in Town',
 'Great Residence Suvarnabhumi Hotel',
 'The 44',
 'Siam Privi Hotel',
 'The Embassy Sathorn',
 'Convenient Park Bangkok Hotel',
 'Tongpoon Hotel',
 'The Aiyapura Bangkok',
 'Nest Boutique Resort',
 '3 Howw Hostel at Sukhumvit 21',
 'Cazz Hostel',
 'Boss Suites Nana',
 '14 Resort',
 'iCheck inn Sukhumvit Soi 2',
 'Centara Grand at Central Plaza Ladprao Bangkok',
 'Arun Haus 39',
 'The Mix Bangkok - Phrom Phong',
 'A House',
 'Le Paradis Bangkok',
 'A&A Guest House',
 'Varinda Hostel',
 'L.A

In [35]:
driver.switch_to_window(default_window_handle)

C:\Users\Paem\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


In [51]:
scroll_height

196

In [56]:
last_height = 0

In [57]:
element_index

1

In [55]:
click_elements

[<selenium.webdriver.remote.webelement.WebElement (session="38a1ac45774fccc217a72abd51bc03e2", element="0.9792992828170923-51")>,
 <selenium.webdriver.remote.webelement.WebElement (session="38a1ac45774fccc217a72abd51bc03e2", element="0.9792992828170923-52")>,
 <selenium.webdriver.remote.webelement.WebElement (session="38a1ac45774fccc217a72abd51bc03e2", element="0.9792992828170923-53")>,
 <selenium.webdriver.remote.webelement.WebElement (session="38a1ac45774fccc217a72abd51bc03e2", element="0.9792992828170923-54")>]

In [60]:
last_height -= 196

In [63]:
len(data_static['hotel_id_in_our_database'])

91

In [65]:
for i in list(data_vary.keys()):
    print(len(data_vary[i]))

872
872
872
872
872
872
872
872
872
872


In [71]:
data_vary['feature_data'][5]

{'Bathroom and toiletries': 'Bathrobes&&Hair dryer&&Mirror&&Scale&&Toiletries&&Towels&&',
 'Bed types': '1 king bed&&',
 'Clothing and laundry': 'Closet&&Clothes rack&&Ironing facilities&&',
 'Comforts': 'Air conditioning&&Linens&&Slippers&&Soundproofing&&Umbrella&&Wake-up service&&',
 'Dining, drinking, and snacking': 'Coffee/tea maker&&Complimentary tea&&Free bottled water&&Free instant coffee&&Microwave&&Mini bar&&Refrigerator&&',
 'Entertainment ': 'DVD/CD player&&Free Wi-Fi in all rooms!&&On-demand movies&&Satellite/cable channels&&Telephone&&',
 'Features': '1 king bed&&Room size: 38 m²/409 ft²&&City view&&Balcony/terrace&&Shower&&Free Wi-Fi&&',
 'Guest ratings': '&&',
 'Layout and furnishings': 'Carpeting&&Desk&&Extra long bed&&Laptop workspace&&',
 'Safety and security features': 'In-room safe box&&Laptop safe box&&Smoke detector&&'}

In [27]:
import pickle
f = open("data_static_19_feb_1.pkl","wb")
pickle.dump(data_static,f)
f.close()
f = open("data_vary_19_feb_1.pkl","wb")
pickle.dump(data_vary,f)
f.close()

In [73]:
driver.close()

In [26]:
len(data_static['hotel_id_in_our_database'])

186

In [5]:
data_vary['hotel_room_Prices_standard']

[5000.0,
 5000.0,
 2446.13,
 2446,
 None,
 2828.46,
 2828,
 3210.79,
 3211,
 3350.82,
 3713.18,
 3593.12,
 3593,
 3803.46,
 3868.0,
 4227.55,
 4255.76,
 4248,
 5995.43,
 4587.17,
 4587,
 5428.29,
 5428,
 5581.22,
 5581,
 5523,
 6117,
 5692,
 6287,
 7009.6,
 7010,
 7172.98,
 8235.51,
 2400,
 2788,
 2560,
 2948,
 2560,
 2948,
 2880,
 3268,
 2960,
 3348,
 5200,
 5975,
 5200,
 5975,
 1942,
 2042,
 333.65,
 327.52,
 352.18,
 345.71,
 None,
 352.18,
 345.71,
 370.72,
 363.91,
 370.72,
 363.91,
 389.26,
 382.1,
 389.26,
 382.1,
 1019.54,
 1020,
 5000.0,
 5508.74,
 6000.0,
 6282.09,
 None,
 6301.58,
 7288.74,
 8000.0,
 8014.45,
 12000.0,
 12000.0,
 2000.0,
 2000.0,
 2000.0,
 2000.0,
 2500.0,
 2500.0,
 2500.0,
 2500.0,
 1472.1,
 1710.25,
 3000.0,
 3000.0,
 1534.45,
 1797.61,
 1885.45,
 2148.6,
 1800.0,
 1799.89,
 651.61,
 595,
 827,
 744.77,
 758.7,
 758.04,
 1033.72,
 2186.89,
 2873.95,
 2300.0,
 2300.0,
 763.02,
 962.06,
 2410.44,
 2399.96,
 862.54,
 1094.77,
 3500.0,
 3500.0,
 765,
 912.08,


In [19]:
element_index

7